## 1. 데이터 전처리

In [98]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [99]:
df=pd.read_csv('data/ArticlesMay2017.csv')
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5906b3197c459f24986dd003,2412,By IAN JOHNSON,article,My Beijing: The Sacred City,"['Travel and Vacations', 'Beijing (China)', 'R...",3,Travel,1,2017-05-01 04:01:21,Unknown,"This metropolis was once a total work of art, ...",The New York Times,News,https://www.nytimes.com/2017/05/01/travel/beij...
1,NaN,5906b3297c459f24986dd004,2318,By EMMA G. FITZSIMMONS,article,"6 Million Riders a Day, 1930s Technology","['Subways', 'Delays (Transportation)', 'Transi...",68,Metro,1,2017-05-01 04:01:33,Unknown,New York’s subway is struggling with old infra...,The New York Times,News,https://www.nytimes.com/2017/05/01/nyregion/ne...
2,NaN,5906ceec7c459f24986dd021,1795,By MARC TRACY,article,Seeking a Cross-Border Conference,"['Cetys University', 'College Athletics', 'Nat...",3,Sports,1,2017-05-01 06:00:05,Unknown,Cetys University is making an ambitious bid to...,The New York Times,News,https://www.nytimes.com/2017/05/01/sports/mexi...
3,NaN,5906cfa77c459f24986dd022,213,By SHANNON DOYNE,article,"Questions for: ‘Despite the “Yuck Factor,” Lee...",[],3,Learning,0,2017-05-01 06:03:03,Unknown,How are leeches used to treat various medical ...,The New York Times,News,https://www.nytimes.com/2017/05/01/learning/qu...
4,NaN,5906e1c07c459f24986dd039,1342,By JASON STANLEY,article,Who Is a ‘Criminal’?,"['Illegal Immigration', 'Traffic and Parking V...",3,OpEd,0,2017-05-01 07:20:26,Unknown,Justice Roberts was right. The Trump administr...,The New York Times,Op-Ed,https://www.nytimes.com/2017/05/01/opinion/who...


In [100]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [101]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [127]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: ‘Despite the “Yuck Factor,” Leeches Are Big in Russian Medicine’',
 'Who Is a ‘Criminal’?']

In [128]:
len(headline)

996

In [129]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

935

In [130]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False    # minus 표시
mpl.rc('font', family='Malgun Gothic')
%matplotlib inline

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["font.size"] = 14

In [131]:
# 구두점 제거와 소문자화를 위한 함수
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii",'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() 

In [132]:
text = [repreprocessing(x) for x in headline]
text[:5]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal']

In [134]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2653


In [135]:
sequences = []
for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[48, 799],
 [48, 799, 1],
 [48, 799, 1, 800],
 [48, 799, 1, 800, 57],
 [114, 406],
 [114, 406, 407],
 [114, 406, 407, 2],
 [114, 406, 407, 2, 179],
 [114, 406, 407, 2, 179, 801],
 [114, 406, 407, 2, 179, 801, 802],
 [803, 2]]

In [136]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len) 

샘플의 최대 길이 : 16


In [137]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len)
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 799]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  48 799   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0  48 799   1 800]]


In [138]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [139]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [140]:
X.shape, y.shape

((5501, 15), (5501, 2653))

## 2. 모델 설계 및 학습

In [141]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [142]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
embedding = Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer")
lstm = LSTM(128, name="LSTM_Layer")
output = Dense(vocab_size, activation='softmax', name="Output_Layer")

In [143]:
model = Sequential()
model.add(embedding)
model.add(lstm)
model.add(output)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 15, 10)            26530     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 2653)              342237    
Total params: 439,935
Trainable params: 439,935
Non-trainable params: 0
_________________________________________________________________


In [144]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [145]:
history = model.fit(X, y, epochs=200, verbose=2) 

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 2s - loss: 7.5105 - accuracy: 0.0314
Epoch 2/200
 - 2s - loss: 6.9964 - accuracy: 0.0353
Epoch 3/200
 - 1s - loss: 6.8710 - accuracy: 0.0371
Epoch 4/200
 - 1s - loss: 6.7740 - accuracy: 0.0407
Epoch 5/200
 - 2s - loss: 6.6590 - accuracy: 0.0478
Epoch 6/200
 - 1s - loss: 6.5201 - accuracy: 0.0487
Epoch 7/200
 - 2s - loss: 6.3649 - accuracy: 0.0538
Epoch 8/200
 - 2s - loss: 6.1986 - accuracy: 0.0567
Epoch 9/200
 - 2s - loss: 6.0242 - accuracy: 0.0607
Epoch 10/200
 - 1s - loss: 5.8548 - accuracy: 0.0707
Epoch 11/200
 - 1s - loss: 5.6920 - accuracy: 0.0725
Epoch 12/200
 - 1s - loss: 5.5366 - accuracy: 0.0767
Epoch 13/200
 - 1s - loss: 5.3921 - accuracy: 0.0816
Epoch 14/200
 - 1s - loss: 5.2525 - accuracy: 0.0887
Epoch 15/200
 - 2s - loss: 5.1202 - accuracy: 0.0916
Epoch 16/200
 - 1s - loss: 4.9893 - accuracy: 0.0983
Epoch 17/200
 - 1s - loss: 4.8673 - accuracy: 0.1027
Epoch 18/200
 - 2s - loss: 4.7476 - accuracy: 0.1107
Epoch 19/200
 - 1s - loss: 4.6329 - accuracy: 0.1240
Ep

Epoch 155/200
 - 2s - loss: 0.4227 - accuracy: 0.8911
Epoch 156/200
 - 2s - loss: 0.4204 - accuracy: 0.8906
Epoch 157/200
 - 2s - loss: 0.4163 - accuracy: 0.8913
Epoch 158/200
 - 2s - loss: 0.4127 - accuracy: 0.8920
Epoch 159/200
 - 2s - loss: 0.4090 - accuracy: 0.8917
Epoch 160/200
 - 2s - loss: 0.4077 - accuracy: 0.8909
Epoch 161/200
 - 2s - loss: 0.4039 - accuracy: 0.8906
Epoch 162/200
 - 2s - loss: 0.4011 - accuracy: 0.8909
Epoch 163/200
 - 2s - loss: 0.3976 - accuracy: 0.8909
Epoch 164/200
 - 2s - loss: 0.4033 - accuracy: 0.8906
Epoch 165/200
 - 2s - loss: 0.4184 - accuracy: 0.8895
Epoch 166/200
 - 2s - loss: 0.4271 - accuracy: 0.8880
Epoch 167/200
 - 2s - loss: 0.4026 - accuracy: 0.8884
Epoch 168/200
 - 2s - loss: 0.3874 - accuracy: 0.8949
Epoch 169/200
 - 2s - loss: 0.3829 - accuracy: 0.8942
Epoch 170/200
 - 2s - loss: 0.3791 - accuracy: 0.8960
Epoch 171/200
 - 2s - loss: 0.3768 - accuracy: 0.8931
Epoch 172/200
 - 2s - loss: 0.3757 - accuracy: 0.8935
Epoch 173/200
 - 2s - loss: 

In [146]:
history.history['accuracy'][199]

0.89547354

## 3. 모델 검증

In [147]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], max_len-1) # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [152]:
df['headline']

0                            My Beijing: The Sacred City
1               6 Million Riders a Day, 1930s Technology
2                      Seeking a Cross-Border Conference
3      Questions for: ‘Despite the “Yuck Factor,” Lee...
4                                   Who Is a ‘Criminal’?
                             ...                        
991    Yes, You Can Write More Than One Letter in a S...
992               Trump’s Act  Of Gratuitous Destruction
993                           Petty Crime, Daunting Bail
994                                              Unknown
995    Bucking Trump, These Cities, States and Compan...
Name: headline, Length: 996, dtype: object

In [154]:
print(sentence_generation(model, t, 'My Beijing', 3))

My Beijing the sacred city


In [159]:
print(sentence_generation(model, t, '6 Million', 5))

6 Million riders a day 1930s technology


In [160]:
print(sentence_generation(model, t, 'Seeking a', 2))

Seeking a crossborder conference
